In [1]:
## Class metaprogramming

class Dog:
    def __init__(self, name, weight, owner):
        self.name = name
        self.weight = weight
        self.owner = owner
        
rex = Dog('Rex', 30, 'Bob')
rex

In [4]:
## record_factory.py
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError:
        pass
    field_names = tuple(field_names)
    
    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
            
    def __iter__(self):
        for name in self.__slots__:
            yield getattr(self, name)
            
    def __repr__(self):
        value = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, value)
    
    cls_attrs = dict(__slots__ = field_names, 
                     __init__  = __init__,
                     __iter__  = __iter__,
                     __repr__  = __repr__)
    
    return type(cls_name, (object,), cls_attrs)

Dog = record_factory('Dog', 'name weight owner')
rex = Dog('Rex', 30, 'Bob')
re

Dog(name='Rex', weight=30, owner='Bob')

In [5]:
name, weight, owner = rex

In [ ]:
# model_v6 
def entity(cls):
    for key, attr in cls.__dict__.items():
        if isinstance(attr, Validated):
            type_name = type(attr).__name__
            attr.storage_name = '_{}#{}'.format(type_name, key)
    return cls


In [7]:
import model_v6 as model

@model.entity
class LineItem:
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [10]:
raisins = LineItem('Golden raisins', 10, 6.95)
dir(raisins)[:3]

['_NonBlank#description', '_Quantity#price', '_Quantity#weight']

In [11]:
## The evaluation time exercises
# evalsupport.py
print('<[100]> evalsupport module start')

def deco_alpha(cls):
    print('<[200]> deco_alpha')
    
    def inner_1(self):
        print('<[300]> deco_alpha:inner_1')
        
    cls.method_y = inner_1
    return cls

#BEGIN META_ALEPH
class MetaAleph(type):
    print('<[400]> MetaAleph body')
    
    def __init__(cls, name, bases, dic):
        print('<[500]> MetaAleph.__init__')
        
        def inner_2(self):
            print('<[600]> MetaAleph.__init__:inner_2')
            
        cls.method_z = inner_2
#END META_ALEPH

print('<[700]> evalsupport module end')

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end


In [1]:
# evaltime.py
from evalsupport import deco_alpha
from evalsupport import MetaAleph

print('<[1]> evaltime module start')

class ClassOne():
    print('<[2]> ClassOne body')
    
    def __init__(self):
        print('<[3]> ClassOne.__init__')
        
    def __del__(self):
        print('<[4]> ClassOne.__del__')
        
    def method_x(self):
        print('<[5]> ClassOne.method_x')
    
    class ClassTwo(object):
        print('<[6]> ClassTwo body')
        
@deco_alpha
class ClassThree():
    print('<[7]> ClassThree body')
    
    def method_y(self):
        print('<[8]> ClassThree.method_y')

class ClassFour(ClassThree):
    print('<[9]> ClassFour body')
    
    def method_y(self):
        print('<[10]> ClassFour.method_y')

class ClassFive(metaclass=MetaAleph):
    print('<[6]> ClassFive body')
    
    def __init__(self):
        print('<[7]> ClassFive.__init__')
        
    def method_z(self):
        print('<[8]> ClassFive.method_y')
        
class ClassSix(ClassFive):
    print('<[9]> ClassSix body')
    
    def method_z(self):
        print('<[10]> ClassSix.method_y')
    
if __name__ == '__main__':
    print('<[11]> ClassOne tests', 30 * '.')
    one = ClassOne()
    one.method_x()
    print('<[12]> ClassThree tests', 30 * '.')
    four = ClassFour()
    four.method_y()
    print('<[13]> ClassFive tests', 30 * '.')
    five = ClassFive()
    five.method_z()
    print('<[14]> ClassSix tests', 30 * '.')
    six = ClassSix()
    six.method_z()

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime module start
<[2]> ClassOne body
<[6]> ClassTwo body
<[7]> ClassThree body
<[200]> deco_alpha
<[9]> ClassFour body
<[6]> ClassFive body
<[500]> MetaAleph.__init__
<[9]> ClassSix body
<[500]> MetaAleph.__init__
<[11]> ClassOne tests ..............................
<[3]> ClassOne.__init__
<[5]> ClassOne.method_x
<[12]> ClassThree tests ..............................
<[10]> ClassFour.method_y
<[13]> ClassFive tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2
<[14]> ClassSix tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2
